In [21]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import glob
from fuzzywuzzy import fuzz, process

In [22]:
parquet_path = "./Staging/Gold/carmotor_scrapped_2_data.parquet"
if os.path.exists(parquet_path):
    New_Leads = pd.read_parquet(parquet_path, engine="fastparquet")
    print(f"Loaded {len(New_Leads)} rows from {parquet_path}")
    print(New_Leads.shape)
else:
    raise FileNotFoundError(f"Parquet file not found at {parquet_path}")

Loaded 514 rows from ./Staging/Gold/carmotor_scrapped_2_data.parquet
(514, 16)


In [23]:
print(New_Leads.shape)
print(New_Leads["UEN"].is_unique)
print(New_Leads["Phones"].is_unique)

(514, 16)
True
True


In [24]:
New_Leads.head(5)

,UEN,Status,Error,Emails,Phones,Website,Facebook,LinkedIn,Instagram,TikTok,RecordOwl_Link,operational_street,operational_unit,operational_postal_code,operational_address,Phones_str
0,202337162C,success,None,[hello@idwrap],['+6580611314'],https://idwrap.sg/,"[https://www.facebook.com/idautowrap/, https:/...",None,None,None,https://recordowl.com/company/id-auto-wrap-pte...,237 ALEXANDRA ROAD THE ALEXCIER,02-05,159929,237 ALEXANDRA ROAD THE ALEXCIER 02-05 Singapor...,None
1,202326625C,success,None,[jaya_automec@yahoo.com],['+6591992566'],None,[https://www.facebook.com/jayaautomec/],None,[https://www.instagram.com/jayaautomec/],None,https://recordowl.com/company/jaya-automec-sg-...,414 ANG MO KIO AVENUE 10 TECK GHEE HEARTLANDS,03-929,560414,414 ANG MO KIO AVENUE 10 TECK GHEE HEARTLANDS ...,None
2,202440698E,success,None,None,['+6588434473'],None,None,None,[https://www.instagram.com/planetperformancesg/],None,https://recordowl.com/company/planet-performan...,60 JALAN LAM HUAT CARROS CENTRE,03-11,737869,60 JALAN LAM HUAT CARROS CENTRE 03-11 Singapor...,None
3,202105832K,success,None,[hydrowashdetailing@gmail.com],['+6580280868'],None,[https://www.facebook.com/Hydrowashdetailing],None,[https://www.instagram.com/hydrowashdetailing/],None,https://recordowl.com/company/hydrowash-detail...,105 BUKIT BATOK CENTRAL NORTH POINT BIZHUB,02-257,650105,105 BUKIT BATOK CENTRAL NORTH POINT BIZHUB 02-...,None
4,202431492H,success,None,None,['+6567416741'],https://1stautogarage.com.sg/,[https://www.facebook.com/1stAutoPteLtdSG/],None,None,None,https://recordowl.com/company/first-automobile...,60 JALAN LAM HUAT CARROS CENTRE,03-42,737869,60 JALAN LAM HUAT CARROS CENTRE 03-42 Singapor...,None


### Restructure to MasterDB Format

In [25]:

Fresh_Leads_format = pd.DataFrame(columns=[
    "ePOS Code",
    "Company Code",
    "Date",
    "ACRA REGISTERED NAME",
    "Brand/Deal Name/Business Name",
    "Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client",
    "Tele Sales or MR (For KPI - Internal)",
    "Name of the Market Researcher",
    "Original Source (Marketing)",
    "Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",
    "Company Registration date / Date Established",
    "Company Registration Number (UEN)",
    "Primary SSIC Code",
    "Secondary SSIC Code",
    "Hubspot ID (Company)",
    "Hubspot ID(Deal)",
    "Hubspot ID(Contact)",
    "Website URL",
    "Business Type",
    "Facebook Page",
    "Instagram URL",
    "Linkedin URL",
    "Tik Tok URL",
    "Ownership Type",
    "Parent Industry Type",
    "Industry Type",
    "Sub Industry",
    "Business model",
    "Presence of Multiple Outlets",
    "Number of Outlets (Write in #)",
    "Region",
    "Planning Area",
    "Business Location Type",
    "Registered Address (Block & Street)",
    "Registered Address  (Unit #)",
    "Registered Address  (Postal code)",
    "Operational Address \n(Block & Street)",
    "Operational Address \n(Unit #)",
    "Operational Address \n(Postal Code)",
    "Operational Address Type",
    "First Name",
    "Last Name",
    "PIC Name 1 Designation",
    "PIC NAME 1 Contact Number",
    "PIC 1 email address",
    "First Name 2",
    "Last Name 2",
    "PIC Name 2 Designation",
    "PIC NAME 2 Contact Number",
    "PIC 2 email address",
    "First Name 3",
    "Last Name 3",
    "PIC Name Designation 3",
    "PIC NAME 3 Contact Number",
    "PIC 3 email address",
    "FB/Insta/Tik Tok/Linkedin Contact",
    "Current ePOS Client ?",
    "If ePOS Client, which product they are using?",
    "Is this deal part of the Gov List?",
    "Source from Market Researcher",
    "Contact Number from Lusha?",
    "Phone number Verified ?"
])


### Mapping scrapped leads to Master DB

In [26]:
# Define mapping from RecordOwl_Leads -> Fresh_Leads_format
cols_map = {
    "UEN": "Company Registration Number (UEN)",
    "Phones": "PIC NAME 1 Contact Number",
    "Emails": "PIC 1 email address",
    "Website": "Website URL",
    "Facebook": "Facebook Page",
    "Instagram": "Instagram URL",
    "TikTok": "Tik Tok URL",
    "LinkedIn": "Linkedin URL",
    "operational_address": "Operational Address \n(Block & Street)",
    "operational_unit": "Operational Address \n(Unit #)",
    "operational_postal_code": "Operational Address \n(Postal Code)",
}

# Columns that contain lists and need to be flattened (extract first element)
# FIXED: Removed "Phones" because it contains strings, not lists
list_columns = ["Emails", "Facebook", "Instagram", "TikTok", "LinkedIn"]

# CRITICAL FIX: Initialize Fresh_Leads_format with the same number of rows as RecordOwl_Leads
Fresh_Leads_format = Fresh_Leads_format.reindex(range(len(New_Leads)))

# Fill Fresh_Leads_format using mapping
for src_col, dest_col in cols_map.items():
    if src_col in New_Leads.columns and dest_col in Fresh_Leads_format.columns:
        # Check if column contains lists
        if src_col in list_columns:
            # Extract first element from list, handle None/NaN gracefully
            Fresh_Leads_format[dest_col] = New_Leads[src_col].apply(
                lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
            ).values
        else:
            # Direct mapping for non-list columns
            Fresh_Leads_format[dest_col] = New_Leads[src_col].values
        print(f"✓ Mapped: {src_col} → {dest_col}")
    else:
        print(f"⚠️ Skipped: {src_col} → {dest_col} (missing in one of the DataFrames)")

✓ Mapped: UEN → Company Registration Number (UEN)
✓ Mapped: Phones → PIC NAME 1 Contact Number
✓ Mapped: Emails → PIC 1 email address
✓ Mapped: Website → Website URL
✓ Mapped: Facebook → Facebook Page
✓ Mapped: Instagram → Instagram URL
✓ Mapped: TikTok → Tik Tok URL
✓ Mapped: LinkedIn → Linkedin URL
✓ Mapped: operational_address → Operational Address 
(Block & Street)
✓ Mapped: operational_unit → Operational Address 
(Unit #)
✓ Mapped: operational_postal_code → Operational Address 
(Postal Code)


In [27]:
# print(Fresh_Leads_format["PIC NAME 1 Contact Number"].dropna())


### Getting ACRA Data and Merging with SSIC Code Mapping & Generating Business Type Column

In [28]:
folder_path = "Acra_Data"

# Get all CSV file paths inside the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and combine all CSVs
df = pd.concat((pd.read_csv(f, low_memory=False) for f in csv_files), ignore_index=True)

df.columns = df.columns.str.upper()

acra_data = df[[
    "UEN",
    "ENTITY_NAME",
    "BUSINESS_CONSTITUTION_DESCRIPTION",
    "ENTITY_TYPE_DESCRIPTION",
    "ENTITY_STATUS_DESCRIPTION",
    "REGISTRATION_INCORPORATION_DATE",
    "PRIMARY_SSIC_CODE",
    "SECONDARY_SSIC_CODE",
    "UNIT_NO",
    "LEVEL_NO",
    "BUILDING_NAME",
    "BLOCK",
    "STREET_NAME",
    "POSTAL_CODE"
]].copy()

# Convert to proper data types
acra_data['UEN'] = acra_data['UEN'].astype('string')
acra_data['ENTITY_NAME'] = acra_data['ENTITY_NAME'].astype('string')
acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'] = acra_data['BUSINESS_CONSTITUTION_DESCRIPTION'].astype('string')
acra_data['ENTITY_TYPE_DESCRIPTION'] = acra_data['ENTITY_TYPE_DESCRIPTION'].astype('string')
acra_data['ENTITY_STATUS_DESCRIPTION'] = acra_data['ENTITY_STATUS_DESCRIPTION'].astype('string')
acra_data['BLOCK'] = acra_data['BLOCK'].astype('string')
acra_data['STREET_NAME'] = acra_data['STREET_NAME'].astype('string')
acra_data['POSTAL_CODE'] = acra_data['POSTAL_CODE'].astype('string')
acra_data['UNIT_NO'] = acra_data['UNIT_NO'].astype('string')
acra_data['LEVEL_NO'] = acra_data['LEVEL_NO'].astype('string')
acra_data['BUILDING_NAME'] = acra_data['BUILDING_NAME'].astype('string')
acra_data['PRIMARY_SSIC_CODE'] = pd.to_numeric(acra_data['PRIMARY_SSIC_CODE'], errors='coerce')
acra_data['SECONDARY_SSIC_CODE'] = pd.to_numeric(acra_data['SECONDARY_SSIC_CODE'], errors='coerce')

# Date column
acra_data['REGISTRATION_INCORPORATION_DATE'] = pd.to_datetime(acra_data['REGISTRATION_INCORPORATION_DATE'], errors='coerce')

# Clean string columns
for col in ['UEN', 'ENTITY_NAME', 'BUSINESS_CONSTITUTION_DESCRIPTION', 'ENTITY_TYPE_DESCRIPTION', 
            'ENTITY_STATUS_DESCRIPTION', 'BLOCK', 'STREET_NAME', 'POSTAL_CODE', 'UNIT_NO', 'LEVEL_NO', 'BUILDING_NAME']:
    acra_data[col] = acra_data[col].fillna('').str.strip().str.replace(r'\s+', ' ', regex=True).str.upper()

acra_data.replace(['NA', 'N/A', '-', ''], np.nan, inplace=True)
acra_data['REGISTRATION_INCORPORATION_DATE'] = acra_data['REGISTRATION_INCORPORATION_DATE'].dt.strftime('%d-%m-%Y')

# Filter only live entities
acra_data = acra_data[acra_data['ENTITY_STATUS_DESCRIPTION'].isin(['LIVE COMPANY', 'LIVE'])].reset_index(drop=True)

# Exclude specific PRIMARY_SSIC_CODE values
exclude_codes = [46900, 47719, 47749, 47539, 47536, 56123, 10711, 10712, 10719, 10732, 10733, 93209]
acra_data = acra_data[~acra_data['PRIMARY_SSIC_CODE'].isin(exclude_codes)].reset_index(drop=True)

# Classify BUSINESS_TYPE - Based on age + ownership type only
reg_date = pd.to_datetime(acra_data['REGISTRATION_INCORPORATION_DATE'], format='%d-%m-%Y', errors='coerce', dayfirst=True)
company_age_years = (pd.Timestamp.today() - reg_date).dt.days / 365.25
ownership_type = acra_data['ENTITY_TYPE_DESCRIPTION'].astype(str)

acra_data['BUSINESS_TYPE'] = np.nan

# Rule 1: age > 3 AND (LOCAL COMPANY | LLP | SOLE PROPRIETORSHIP) → SME
mask = (company_age_years > 3) & ownership_type.str.contains('LOCAL COMPANY|LIMITED LIABILITY PARTNERSHIP|SOLE PROPRIETORSHIP/ PARTNERSHIP', case=False, na=False)
acra_data.loc[mask, 'BUSINESS_TYPE'] = 'SME'

# Rule 2: age > 5 AND FOREIGN COMPANY BRANCH → Large Enterprise
mask = (company_age_years > 5) & ownership_type.str.contains('FOREIGN COMPANY BRANCH', case=False, na=False) & acra_data['BUSINESS_TYPE'].isna()
acra_data.loc[mask, 'BUSINESS_TYPE'] = 'Large Enterprise'

# Rule 3: age > 5 AND (LOCAL COMPANY | LLP) → Franchise
mask = (company_age_years > 5) & ownership_type.str.contains('LOCAL COMPANY|LIMITED LIABILITY PARTNERSHIP', case=False, na=False) & acra_data['BUSINESS_TYPE'].isna()
acra_data.loc[mask, 'BUSINESS_TYPE'] = 'Franchise'

# Rule 4: age < 5 AND SOLE PROPRIETORSHIP → Startup
mask = (company_age_years < 5) & ownership_type.str.contains('SOLE PROPRIETORSHIP/ PARTNERSHIP', case=False, na=False) & acra_data['BUSINESS_TYPE'].isna()
acra_data.loc[mask, 'BUSINESS_TYPE'] = 'Startbud'

# Fallback rules based on age only
# Startup: Age ≤ 4
# SME: Age 5–10
# Large Enterprise: Age ≥ 11 OR (Foreign Company Branch with Age ≥ 7)
unclassified = acra_data['BUSINESS_TYPE'].isna()
acra_data.loc[unclassified & (company_age_years >= 11), 'BUSINESS_TYPE'] = 'Large Enterprise'
acra_data.loc[unclassified & (company_age_years >= 7) & ownership_type.str.contains('FOREIGN COMPANY BRANCH', case=False, na=False), 'BUSINESS_TYPE'] = 'Large Enterprise'
acra_data.loc[unclassified & (company_age_years >= 5) & (company_age_years <= 10) & acra_data['BUSINESS_TYPE'].isna(), 'BUSINESS_TYPE'] = 'SME'
acra_data.loc[unclassified & (company_age_years <= 4) & acra_data['BUSINESS_TYPE'].isna(), 'BUSINESS_TYPE'] = 'Startbud'

# Getting SSIC Code
file_path = "./SSIC_Code/mapped_ssic_code.xlsx"
mapped_ssic_code = pd.read_excel(file_path)
mapped_ssic_code.columns = mapped_ssic_code.columns.str.strip().str.upper().str.replace(" ", "_")
columns_to_keep = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "SSIC_CODES", "DESCRIPTION"]
mapped_ssic_code = mapped_ssic_code[columns_to_keep].copy()
mapped_ssic_code["SSIC_CODES"] = pd.to_numeric(mapped_ssic_code["SSIC_CODES"], errors="coerce").fillna(0).astype(int)
text_cols = ["PARENT_INDUSTRY", "INDUSTRY_TYPE", "SUB_INDUSTRY", "DESCRIPTION"]
mapped_ssic_code[text_cols] = mapped_ssic_code[text_cols].apply(lambda col: col.astype(str).str.strip().str.title())
mapped_ssic_code = mapped_ssic_code.drop_duplicates().reset_index(drop=True)

# Merging
acra_data["PRIMARY_SSIC_CODE"] = pd.to_numeric(acra_data["PRIMARY_SSIC_CODE"], errors="coerce").fillna(0).astype(int)
acra_data["SECONDARY_SSIC_CODE"] = pd.to_numeric(acra_data["SECONDARY_SSIC_CODE"], errors="coerce").fillna(0).astype(int)
acra_data_mapped = acra_data.merge(mapped_ssic_code, how="left", left_on="PRIMARY_SSIC_CODE", right_on="SSIC_CODES")
acra_data_mapped = acra_data_mapped.drop(columns=["SSIC_CODES"], errors="ignore")

print("\n✅ ACRA Business Type classification completed!")
print(f"\nBusiness Type distribution:\n{acra_data_mapped['BUSINESS_TYPE'].value_counts(dropna=False)}")


C:\Users\azlie\AppData\Local\Temp\ipykernel_15972\3220291497.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SME' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  acra_data.loc[mask, 'BUSINESS_TYPE'] = 'SME'



✅ ACRA Business Type classification completed!

Business Type distribution:
BUSINESS_TYPE
SME                 380246
Startbud            155492
Large Enterprise      1438
NaN                    152
Name: count, dtype: int64


In [29]:
acra_data_mapped.shape

(537328, 19)

### Mapping Leads with ACRA Data

In [30]:
# Merge with ACRA data
merged = Fresh_Leads_format.merge(
    acra_data_mapped[[
        "UEN",
        "ENTITY_NAME",
        "PRIMARY_SSIC_CODE",
        "SECONDARY_SSIC_CODE",
        "REGISTRATION_INCORPORATION_DATE",
        "ENTITY_TYPE_DESCRIPTION",
        "BUSINESS_TYPE",
        "PARENT_INDUSTRY",
        "INDUSTRY_TYPE",
        "SUB_INDUSTRY",
        "BLOCK",
        "STREET_NAME",
        "UNIT_NO",
        "POSTAL_CODE"
    ]],
    left_on="Company Registration Number (UEN)",
    right_on="UEN",
    how="left"
)

# Create combined address fields
merged['REGISTERED_ADDRESS_COMBINED'] = (
    merged['BLOCK'].fillna('').astype(str) + ' ' + 
    merged['STREET_NAME'].fillna('').astype(str)
).str.strip().replace('', np.nan)

# Map ACRA data to Fresh_Leads_format columns
merged["ACRA REGISTERED NAME"] = merged["ENTITY_NAME"]
merged["Brand/Deal Name/Business Name"] = merged["ENTITY_NAME"]
merged["Primary SSIC Code"] = merged["PRIMARY_SSIC_CODE"]
merged["Secondary SSIC Code"] = merged["SECONDARY_SSIC_CODE"]
merged["Company Registration date / Date Established"] = merged["REGISTRATION_INCORPORATION_DATE"]
merged["Ownership Type"] = merged["ENTITY_TYPE_DESCRIPTION"]
merged["Business Type"] = merged["BUSINESS_TYPE"]
merged["Parent Industry Type"] = merged["PARENT_INDUSTRY"]
merged["Industry Type"] = merged["INDUSTRY_TYPE"]
merged["Sub Industry"] = merged["SUB_INDUSTRY"]
merged["Registered Address (Block & Street)"] = merged['REGISTERED_ADDRESS_COMBINED']
merged["Registered Address  (Unit #)"] = merged["UNIT_NO"]
merged["Registered Address  (Postal code)"] = merged["POSTAL_CODE"]
merged["Operational Address \n(Unit #)"] = merged["UNIT_NO"]
merged["Operational Address \n(Postal Code)"] = merged["POSTAL_CODE"]

# Keep ONLY the original Fresh_Leads_format columns and make a proper copy
Fresh_Leads_formatted = merged[Fresh_Leads_format.columns].copy()

### EPOS Backend Confirmation

In [32]:
epos_backend_df = pd.read_csv(
    "./Epos_Backend/organizations_export.csv",
    on_bad_lines="skip"  # skips rows with too many or too few fields
)

epos_backend_df = epos_backend_df.loc[
    epos_backend_df["status"] == "Active",
    ["organization_name", "status"]
]

epos_backend_df

# Initialize "Current ePOS Client ?" column with "No" for all rows
Fresh_Leads_formatted['Current ePOS Client ?'] = 'No'

# Clean and prepare the data for matching
# Convert to string and strip, but keep all rows (including NaN)
Fresh_Leads_formatted['ACRA REGISTERED NAME_cleaned'] = Fresh_Leads_formatted['ACRA REGISTERED NAME'].astype(str).str.strip()
epos_backend_names = epos_backend_df['organization_name'].dropna().astype(str).str.strip().tolist()

# Set similarity threshold (100% match - exact match only)
THRESHOLD = 100

# Track matches
matches_found = []
matched_indices = []

# Get rows with valid (non-null, non-nan) names
valid_name_mask = (
    Fresh_Leads_formatted['ACRA REGISTERED NAME'].notna() & 
    (Fresh_Leads_formatted['ACRA REGISTERED NAME_cleaned'] != 'nan') &
    (Fresh_Leads_formatted['ACRA REGISTERED NAME_cleaned'] != '')
)

valid_rows = Fresh_Leads_formatted[valid_name_mask]
print(f"Checking {len(valid_rows)} names against {len(epos_backend_names)} ePOS backend organizations...\n")
print("=" * 80)

# For each row with a valid name, check if it exists in ePOS backend
for idx, row in valid_rows.iterrows():
    name = row['ACRA REGISTERED NAME_cleaned']
    
    # Find the best match using fuzzy matching
    best_match = process.extractOne(
        name, 
        epos_backend_names,
        scorer=fuzz.token_sort_ratio
    )
    
    if best_match and best_match[1] >= THRESHOLD:
        matches_found.append({
            'Fresh Lead Name': name,
            'Matched ePOS Name': best_match[0],
            'Similarity Score': best_match[1]
        })
        matched_indices.append(idx)  # Store the original index
        # Update the dataframe: set "Current ePOS Client ?" to "Yes" for this row
        Fresh_Leads_formatted.at[idx, 'Current ePOS Client ?'] = 'Yes'
        print(f"yes there's exist!")
        print(f"  Fresh Lead: {name}")
        print(f"  Matched with: {best_match[0]}")
        print(f"  Similarity: {best_match[1]}%")
        print("-" * 80)

# Create Epos_Client_df from matched rows (those with 100% similarity)
Epos_Client_df = Fresh_Leads_formatted.loc[matched_indices].copy()

# Drop matched rows from Fresh_Leads_formatted
Fresh_Leads_formatted = Fresh_Leads_formatted.drop(index=matched_indices)

# Drop the temporary cleaned column from both dataframes
Fresh_Leads_formatted = Fresh_Leads_formatted.drop(columns=['ACRA REGISTERED NAME_cleaned'])
Epos_Client_df = Epos_Client_df.drop(columns=['ACRA REGISTERED NAME_cleaned'])

# Reset indices
Fresh_Leads_formatted = Fresh_Leads_formatted.reset_index(drop=True)
Epos_Client_df = Epos_Client_df.reset_index(drop=True)

print(f"\n\nTotal matches found: {len(matches_found)} out of {len(valid_rows)}")
print(f"Rows transferred to Epos_Client_df: {len(Epos_Client_df)}")
print(f"Rows remaining in Fresh_Leads_formatted: {len(Fresh_Leads_formatted)}")

# Create a summary DataFrame
if matches_found:
    matches_df = pd.DataFrame(matches_found)
    print("\nSummary of matches:")
    print(matches_df)
else:
    print("\nNo matches found above the threshold.")

# Display both dataframes
print("\n\nEpos_Client_df (existing clients with 100% match):")
print(Epos_Client_df[['ACRA REGISTERED NAME', 'Current ePOS Client ?']].head(20))

print("\n\nFresh_Leads_formatted (remaining leads):")
print(Fresh_Leads_formatted[['ACRA REGISTERED NAME', 'Current ePOS Client ?']].head(20))

Checking 514 names against 2343 ePOS backend organizations...

yes there's exist!
  Fresh Lead: HALO COLLECTIVE PTE. LTD.
  Matched with: HALO COLLECTIVE PTE. LTD.
  Similarity: 100%
--------------------------------------------------------------------------------


Total matches found: 1 out of 514
Rows transferred to Epos_Client_df: 1
Rows remaining in Fresh_Leads_formatted: 513

Summary of matches:
             Fresh Lead Name          Matched ePOS Name  Similarity Score
0  HALO COLLECTIVE PTE. LTD.  HALO COLLECTIVE PTE. LTD.               100


Epos_Client_df (existing clients with 100% match):
        ACRA REGISTERED NAME Current ePOS Client ?
0  HALO COLLECTIVE PTE. LTD.                   Yes


Fresh_Leads_formatted (remaining leads):
                      ACRA REGISTERED NAME Current ePOS Client ?
0                   ID AUTO WRAP PTE. LTD.                    No
1                JAYA AUTOMEC SG PTE. LTD.                    No
2             PLANET PERFORMANCE PTE. LTD.             

In [33]:
Epos_Client_df

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,NaN,HALO COLLECTIVE PTE. LTD.,HALO COLLECTIVE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN


In [34]:
Fresh_Leads_formatted

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,NaN,ID AUTO WRAP PTE. LTD.,ID AUTO WRAP PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,JAYA AUTOMEC SG PTE. LTD.,JAYA AUTOMEC SG PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,PLANET PERFORMANCE PTE. LTD.,PLANET PERFORMANCE PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,HYDROWASH DETAILING PTE. LTD.,HYDROWASH DETAILING PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,FIRST AUTOMOBILE SERVICES PTE. LTD.,FIRST AUTOMOBILE SERVICES PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,NaN,NaN,NaN,DETAILOUT PTE. LTD.,DETAILOUT PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
509,NaN,NaN,NaN,CRAFTER WERK,CRAFTER WERK,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
510,NaN,NaN,NaN,HKL LIM TEAM MOTORSPORT PTE. LTD.,HKL LIM TEAM MOTORSPORT PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN
511,NaN,NaN,NaN,NCS MOTORWERKZ PTE. LTD.,NCS MOTORWERKZ PTE. LTD.,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN


### Prefil Data

In [ ]:
Fresh_Leads_formatted['Date'] = pd.Timestamp.today().normalize()
Fresh_Leads_formatted['Date'] = Fresh_Leads_formatted['Date'].dt.strftime("%d,%m,%Y")

# fill specific columns with given values
Fresh_Leads_formatted["Tele Sales or MR (For KPI - Internal)"] = "TeleSales"
Fresh_Leads_formatted["Name of the Market Researcher"] = "Jasmin"
Fresh_Leads_formatted["Original Source (Marketing)"] = "Offline Sources"
Fresh_Leads_formatted["Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)"] = "Web Scrapping"
Fresh_Leads_formatted["Is this deal part of the Gov List?"] = "Gov List"
Fresh_Leads_formatted["Contact Number from Lusha?"] = "No"
Fresh_Leads_formatted["Source from Market Researcher"] = [["ACRA", "Google Searches"]] * len(Fresh_Leads_formatted)
Fresh_Leads_formatted["Business model"] = [["B2C", "Offline"]] * len(Fresh_Leads_formatted)
Fresh_Leads_formatted["Contact Number from Lusha?"] = "No"

### Cleaning & Reformat

In [36]:
# ============================================================================
# COMPREHENSIVE DATA CLEANING MODULE
# ============================================================================

# ----------------------------------------------------------------------------
# 1. COMPANY NAME CLEANING
# ----------------------------------------------------------------------------

def clean_company_name(name):
    """
    Remove common company suffixes and clean company names.
    Handles Singapore, Malaysia, and international suffixes.
    """
    if pd.isna(name) or str(name).strip() == '':
        return name
    
    name = str(name).strip()
    
    # Remove common company suffixes (expanded list)
    name = re.sub(
        r'\b(PTE\.?\s*LTD\.?|PTE\.?|LTD\.?|LIMITED|SDN\.?\s*BHD\.?|SDN\.?|BHD\.?|'
        r'INC\.?|INCORPORATED|CORP\.?|CORPORATION|LLP|LLC|PLC|'
        r'CO\.?|COMPANY|COMPANIES|ENTERPRISE|ENTERPRISES|TRADING|'
        r'PLT|SINGAPORE|SG|HOLDINGS?|HOLDING|GROUP|'
        r'PRIVATE|PUBLIC|INTERNATIONAL|INTL\.?|GLOBAL|'
        r'SERVICES?|SOLUTIONS?|SYSTEMS?|TECHNOLOGIES|TECH)\b',
        '', name, flags=re.IGNORECASE
    )
    
    # Remove extra punctuation but keep meaningful characters (hyphen at end to avoid range error)
    name = re.sub(r'[^\w\s&@#+\-]', '', name)  # Keep alphanumeric, &, @, #, +, -, spaces
    
    # Remove multiple spaces
    name = re.sub(r'\s+', ' ', name).strip()
    
    # Remove trailing/leading special characters (hyphen at end)
    name = re.sub(r'^[&@#+\-\s]+|[&@#+\-\s]+$', '', name).strip()
    
    return name if name else pd.NA


# ----------------------------------------------------------------------------
# 2. PHONE NUMBER CLEANING & FORMATTING
# ----------------------------------------------------------------------------

def clean_phone_number(phone):
    """
    Clean and format phone numbers to Singapore standard: 65 XXXX XXXX
    Handles various input formats and validates phone numbers.
    """
    if pd.isna(phone) or str(phone).strip() in ['', 'nan', 'None', 'null']:
        return pd.NA
    
    # Convert to string and extract all digits
    digits = re.sub(r'\D', '', str(phone))
    
    if not digits:
        return pd.NA
    
    # Remove leading zeros
    digits = digits.lstrip('0')
    
    # Handle country codes
    if digits.startswith('65'):
        # Singapore number with country code
        local_part = digits[2:]
        if len(local_part) >= 8:
            # Format: 65 XXXX XXXX
            return f"65 {local_part[:4]} {local_part[4:8]}"
        elif len(local_part) >= 4:
            return f"65 {local_part}"
        else:
            return pd.NA  # Invalid Singapore number
    
    elif len(digits) == 8:
        # Local Singapore number without country code
        return f"65 {digits[:4]} {digits[4:]}"
    
    elif len(digits) == 10 and digits.startswith('65'):
        # 65XXXXXXXX format
        return f"65 {digits[2:6]} {digits[6:10]}"
    
    elif len(digits) > 8:
        # Assume first part is country code, take last 8 digits as local number
        local_part = digits[-8:]
        return f"65 {local_part[:4]} {local_part[4:]}"
    
    else:
        # Less than 8 digits - likely invalid or incomplete
        return pd.NA


# ----------------------------------------------------------------------------
# 3. EMAIL CLEANING
# ----------------------------------------------------------------------------

def clean_email(email):
    """
    Clean and validate email addresses.
    Remove brackets, quotes, and validate format.
    """
    if pd.isna(email) or str(email).strip() in ['', 'nan', 'None', 'null']:
        return pd.NA
    
    email = str(email).strip()
    
    # Remove brackets, quotes, and other unwanted characters
    email = re.sub(r'[\[\]\'\"+,]', '', email)
    email = email.strip().lower()
    
    # Basic email validation
    if re.match(r'^[a-z0-9][a-z0-9._-]*@[a-z0-9][a-z0-9.-]+\.[a-z]{2,}$', email):
        return email
    else:
        return pd.NA


# ----------------------------------------------------------------------------
# 4. URL CLEANING (for social media and websites)
# ----------------------------------------------------------------------------

def clean_url(url):
    """
    Clean URLs by removing brackets, quotes, and validating format.
    """
    if pd.isna(url) or str(url).strip() in ['', 'nan', 'None', 'null']:
        return pd.NA
    
    url = str(url).strip()
    
    # Remove brackets, quotes, commas
    url = re.sub(r'[\[\]\'\"+,]', '', url)
    url = url.strip()
    
    # Basic URL validation (starts with http/https or www or common domains)
    if re.match(r'^(https?://|www\.|[a-z]+\.(com|sg|net|org|co))', url, re.IGNORECASE):
        return url
    else:
        return pd.NA


# ----------------------------------------------------------------------------
# 5. LIST/ARRAY COLUMN CLEANING
# ----------------------------------------------------------------------------

def clean_list_column(value):
    """
    Clean columns that contain list/array values.
    Handles both actual Python lists and stringified lists.
    Returns as comma-separated string.
    """
    # Handle actual None/NaN values
    if value is None:
        return pd.NA
    
    # Check if it's already a pandas NA
    try:
        if pd.isna(value):
            return pd.NA
    except (ValueError, TypeError):
        # If pd.isna() fails (for lists), continue processing
        pass
    
    # If it's already a list, convert to comma-separated string
    if isinstance(value, list):
        if len(value) == 0:
            return pd.NA
        # Clean each item in the list
        cleaned_items = [str(item).strip() for item in value if item is not None and str(item).strip() not in ['', 'nan', 'None', 'null']]
        if cleaned_items:
            return ', '.join(cleaned_items)
        else:
            return pd.NA
    
    # Convert to string for string-based processing
    value_str = str(value).strip()
    
    # Check for empty or null strings
    if value_str in ['', 'nan', 'None', 'null']:
        return pd.NA
    
    # Remove brackets and quotes from stringified lists
    value_str = re.sub(r'[\[\]\'\"+]', '', value_str)
    
    # Clean up commas and spaces
    value_str = re.sub(r'\s*,\s*', ', ', value_str).strip()
    
    return value_str if value_str else pd.NA


# ----------------------------------------------------------------------------
# 6. APPLY CLEANING TO ALL COLUMNS (SILENT MODE)
# ----------------------------------------------------------------------------

# Clean Company/Brand Names (ONLY Brand/Deal Name, NOT ACRA REGISTERED NAME)
if "Brand/Deal Name/Business Name" in Fresh_Leads_formatted.columns:
    Fresh_Leads_formatted["Brand/Deal Name/Business Name"] = Fresh_Leads_formatted["Brand/Deal Name/Business Name"].apply(clean_company_name)

# Clean Phone Numbers
phone_columns = ["PIC NAME 1 Contact Number", "PIC NAME 2 Contact Number", "PIC NAME 3 Contact Number", "Contact Number from Lusha?"]
for col in phone_columns:
    if col in Fresh_Leads_formatted.columns:
        Fresh_Leads_formatted[col] = Fresh_Leads_formatted[col].apply(clean_phone_number)

# Clean Emails
email_columns = ["PIC 1 email address", "PIC 2 email address", "PIC 3 email address"]
for col in email_columns:
    if col in Fresh_Leads_formatted.columns:
        Fresh_Leads_formatted[col] = Fresh_Leads_formatted[col].apply(clean_email)

# Clean URLs (Social Media & Websites)
url_columns = ["Website URL", "Facebook Page", "Instagram URL", "Linkedin URL", "Tik Tok URL", 
               "Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client"]
for col in url_columns:
    if col in Fresh_Leads_formatted.columns:
        Fresh_Leads_formatted[col] = Fresh_Leads_formatted[col].apply(clean_url)

# Clean List/Array Columns
list_columns = ["Source from Market Researcher", "Business model", "FB/Insta/Tik Tok/Linkedin Contact"]
for col in list_columns:
    if col in Fresh_Leads_formatted.columns:
        Fresh_Leads_formatted[col] = Fresh_Leads_formatted[col].apply(clean_list_column)

# Clean Text Columns (General)
text_columns = ["First Name", "Last Name", "First Name 2", "Last Name 2", "First Name 3", "Last Name 3",
                "PIC Name 1 Designation", "PIC Name 2 Designation", "PIC Name Designation 3",
                "Name of the Market Researcher", "Tele Sales or MR (For KPI - Internal)"]
for col in text_columns:
    if col in Fresh_Leads_formatted.columns:
        Fresh_Leads_formatted[col] = Fresh_Leads_formatted[col].apply(
            lambda x: str(x).strip() if pd.notna(x) and str(x).strip() not in ['', 'nan', 'None'] else pd.NA
        )


# ----------------------------------------------------------------------------
# 7. CORE INDICATORS - CLEANING STATUS & DUPLICATES
# ----------------------------------------------------------------------------

print("=" * 70)
print("📋 DATA CLEANING SUMMARY")
print("=" * 70)

# Check for duplicate UENs
uen_has_duplicates = False
if "Company Registration Number (UEN)" in Fresh_Leads_formatted.columns:
    uen_duplicates = Fresh_Leads_formatted[Fresh_Leads_formatted.duplicated(subset=["Company Registration Number (UEN)"], keep=False)]
    uen_dup_count = len(uen_duplicates)
    if uen_dup_count > 0:
        uen_has_duplicates = True
        print(f"⚠️  UEN Duplicates: {uen_dup_count} rows ({uen_duplicates['Company Registration Number (UEN)'].nunique()} unique UENs)")
    else:
        print(f"✅ UEN Duplicates: None")

# Check for duplicate Phone Numbers
phone_has_duplicates = False
if "PIC NAME 1 Contact Number" in Fresh_Leads_formatted.columns:
    phone_duplicates = Fresh_Leads_formatted[
        Fresh_Leads_formatted.duplicated(subset=["PIC NAME 1 Contact Number"], keep=False) &
        Fresh_Leads_formatted["PIC NAME 1 Contact Number"].notna()
    ]
    phone_dup_count = len(phone_duplicates)
    if phone_dup_count > 0:
        phone_has_duplicates = True
        print(f"⚠️  Phone Duplicates: {phone_dup_count} rows ({phone_duplicates['PIC NAME 1 Contact Number'].nunique()} unique numbers)")
    else:
        print(f"✅ Phone Duplicates: None")

# Cleaning completion status
print(f"\n✅ All columns cleaned successfully")
print(f"   (Excluded: ACRA REGISTERED NAME, UEN)")

print("=" * 70)

📋 DATA CLEANING SUMMARY
✅ UEN Duplicates: None
⚠️  Phone Duplicates: 12 rows (6 unique numbers)

✅ All columns cleaned successfully
   (Excluded: ACRA REGISTERED NAME, UEN)


In [37]:
Fresh_Leads_formatted.shape

(513, 62)

In [38]:
Fresh_Leads_formatted

,ePOS Code,Company Code,Date,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,Sub Domain Link (If Lead is already available in Backend) Fill only when EPOS client,Tele Sales or MR (For KPI - Internal),Name of the Market Researcher,Original Source (Marketing),"Marketing Source (Do not fill anything if the leads are from Hubspot, EPOS clients)",...,PIC Name Designation 3,PIC NAME 3 Contact Number,PIC 3 email address,FB/Insta/Tik Tok/Linkedin Contact,Current ePOS Client ?,"If ePOS Client, which product they are using?",Is this deal part of the Gov List?,Source from Market Researcher,Contact Number from Lusha?,Phone number Verified ?
0,NaN,NaN,"03,12,2025",ID AUTO WRAP PTE. LTD.,ID AUTO WRAP,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
1,NaN,NaN,"03,12,2025",JAYA AUTOMEC SG PTE. LTD.,JAYA AUTOMEC,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
2,NaN,NaN,"03,12,2025",PLANET PERFORMANCE PTE. LTD.,PLANET PERFORMANCE,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
3,NaN,NaN,"03,12,2025",HYDROWASH DETAILING PTE. LTD.,HYDROWASH DETAILING,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
4,NaN,NaN,"03,12,2025",FIRST AUTOMOBILE SERVICES PTE. LTD.,FIRST AUTOMOBILE,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,NaN,NaN,"03,12,2025",DETAILOUT PTE. LTD.,DETAILOUT,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
509,NaN,NaN,"03,12,2025",CRAFTER WERK,CRAFTER WERK,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
510,NaN,NaN,"03,12,2025",HKL LIM TEAM MOTORSPORT PTE. LTD.,HKL LIM TEAM MOTORSPORT,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN
511,NaN,NaN,"03,12,2025",NCS MOTORWERKZ PTE. LTD.,NCS MOTORWERKZ,<NA>,TeleSales,Jasmin,Offline Sources,Web Scrapping,...,<NA>,<NA>,<NA>,<NA>,No,NaN,Gov List,"ACRA, Google Searches",<NA>,NaN


In [ ]:
Fresh_Leads_formatted.to_parquet("./Staging/Gold/cleaned_second_592.parquet", index=False, engine="fastparquet")

### Examine data

In [45]:
x = Fresh_Leads_formatted[[
    "ACRA REGISTERED NAME",
    "Brand/Deal Name/Business Name",
    "PIC NAME 1 Contact Number",
    "Business Type",
    "Registered Address (Block & Street)", 
    "Registered Address  (Unit #)", 
    "Registered Address  (Postal code)", 
    "Operational Address \n(Postal Code)",
]]
x.head(50)


,ACRA REGISTERED NAME,Brand/Deal Name/Business Name,PIC NAME 1 Contact Number,Business Type,Registered Address (Block & Street),Registered Address (Unit #),Registered Address (Postal code),Operational Address \n(Postal Code)
0,ID AUTO WRAP PTE. LTD.,ID AUTO WRAP,65 8061 1314,Startbud,237 ALEXANDRA ROAD,05,159929,159929
1,JAYA AUTOMEC SG PTE. LTD.,JAYA AUTOMEC,65 9199 2566,Startbud,414 ANG MO KIO AVENUE 10,929,560414,560414
2,PLANET PERFORMANCE PTE. LTD.,PLANET PERFORMANCE,65 8843 4473,Startbud,60 JALAN LAM HUAT,11,737869,737869
3,HYDROWASH DETAILING PTE. LTD.,HYDROWASH DETAILING,65 8028 0868,SME,2 YISHUN INDUSTRIAL STREET 1,01,768159,768159
4,FIRST AUTOMOBILE SERVICES PTE. LTD.,FIRST AUTOMOBILE,65 6741 6741,Startbud,60 JALAN LAM HUAT,42,737869,737869
5,VIP CONCEPT,VIP CONCEPT,65 9069 3800,SME,498B TAMPINES STREET 45,380,521498,521498
6,REWORK DETAILING PTE. LTD.,REWORK DETAILING,65 9639 6966,Startbud,1 CORPORATION DRIVE,10,619775,619775
7,ELITE AUTO MOTORS PTE. LTD.,ELITE AUTO MOTORS,65 8368 1036,SME,2 YISHUN INDUSTRIAL STREET 1,06,768159,768159
8,AUTO-TECHNICA PRIVATE LIMITED,AUTO-TECHNICA,65 6214 0964,SME,20 CHANGI NORTH CRESCENT,00,499613,499613
9,TECH-HUB SERVICES PTE. LTD.,HUB,65 6409 9672,Startbud,220 ORCHARD ROAD,02,238852,238852
